In [1]:
%%capture
!pip install folium matplotlib mapclassify

In [16]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Nils

In [17]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


# Import the percentage of students that attend a primary schools located in a NIL different from the one they reside.

Only NILs with more than 15000 inhabitants are considered

In [51]:
pe = pd.read_csv(os.path.join(FOLDER_PATH, 'primaria_iscrizione_nil_2009_2022.csv'),sep=';')
pe = pe.rename(columns={'Anno scolastico':'school_year',
                        'ID NIL':'ID_NIL',
                        'Percentuale studenti':'pupils_perc'})

pe = pe.drop('NIL ',axis=1)

pe['ID_NIL'] = pe['ID_NIL'].astype(str)
pe

,school_year,ID_NIL,pupils_perc
0,2009/10,1,53.6
1,2009/10,2,41
2,2009/10,4,46.9
3,2009/10,6,67.9
4,2009/10,7,34.9
...,...,...,...
567,2021/22,70,"100,0"
568,2021/22,71,"37,9"
569,2021/22,76,"34,6"
570,2021/22,79,"37,6"


In [52]:
pe['school_year']=pe['school_year'].str.split('/', expand=True)[0]
pe = pe[pe['school_year']=='2021'].drop('school_year',axis=1)
pe['pupils_perc'] = pe['pupils_perc'].str.replace(',','.').astype(float)
pe

,ID_NIL,pupils_perc
528,1,61.2
529,2,53.4
530,4,52.9
531,6,68.7
532,7,41.9
533,10,68.4
534,11,54.4
535,12,39.5
536,13,43.9
537,14,33.0


In [56]:
primary_schools = nil.merge(pe, how='left', on='ID_NIL').fillna(0).drop('geometry',axis=1)
primary_schools

,ID_NIL,NIL,area_sqkm,Residenti,pupils_perc
0,1,duomo,2.341616,16608,61.2
1,2,brera,1.637333,18294,53.4
2,3,giardini p.ta venezia,0.249637,43,0.0
3,4,guastalla,1.547962,15528,52.9
4,5,porta vigentina - porta lodovica,1.135196,13548,0.0
...,...,...,...,...,...
83,84,parco nord,1.532331,98,0.0
84,85,parco delle abbazie,13.733841,364,0.0
85,86,parco dei navigli,3.623149,354,0.0
86,87,assiano,5.840942,211,0.0


In [57]:
#primary_schools.to_csv(os.path.join(FOLDER_PATH, 'Processed','primary_schools.csv'),index=False)